In [46]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

## 데이터 확인

In [2]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [47]:
# 파일 불러오기

train = pd.read_csv('./open/train.csv', encoding='utf-8')
test_x = pd.read_csv('./open/test_x.csv', encoding='utf-8')
submission = pd.read_csv('./open/sample_submission.csv', encoding='utf-8')

### train data
- 실제 작가와 작가가 작성한 Text가 라벨링된 데이터프레임

In [4]:
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [5]:
train.author.unique(), train.author.value_counts()

(array([3, 2, 1, 4, 0]), 3    15063
 0    13235
 2    11554
 4     7805
 1     7222
 Name: author, dtype: int64)

### text_x
- train data를 학습시켜 어떤 작가가 작성했는지 분석해야할 데이터프레임

In [6]:
test_x

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...
...,...,...
19612,19612,"At the end of another day or two, odin growing..."
19613,19613,"All afternoon we sat together, mostly in silen..."
19614,19614,"odin, having carried his thanks to odin, proc..."
19615,19615,"Soon after this, upon odin's leaving the room,..."


### submission
- 제출 형태
- 가로 index : 작가 명

In [7]:
submission

,index,0,1,2,3,4
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0
...,...,...,...,...,...,...
19612,19612,0,0,0,0,0
19613,19613,0,0,0,0,0
19614,19614,0,0,0,0,0
19615,19615,0,0,0,0,0


## 전처리

### nltk.stopwords + train_test_split

In [1]:
X = pd.read_csv('open/word.csv')
X.text = X.text.astype('str')
X

,text,author
0,he was almost choking there was so much so muc...,3
1,your sister asked for it i suppose,2
2,she was engaged one day as she walked in perus...,1
3,the captain was in the porch keeping himself c...,4
4,have mercy gentlemen odin flung up his hands ...,3
...,...,...
54874,is that you mr smith odin whispered i hardly ...,2
54875,i told my plan to the captain and between us w...,4
54876,your sincere well wisher friend and sister luc...,1
54877,then you wanted me to lend you money,3


In [48]:
x0 = train[train['author'] == 0].reset_index(drop=True)
x00 = x0.sample(n=5000, random_state=13).reset_index(drop=True)
x00

,index,text,author
0,31768,"The suitor, kissing the hem of the garment aga...",0
1,21061,"“At,” repeated Mr. odin, still looking at the ...",0
2,10750,"“Dear, dear! Give it me back, Mum,” said Flops...",0
3,47317,"‘Be calm, my dear ma’am,’ said Mr. Chillip, in...",0
4,16917,Because he had voluntarily relinquished a titl...,0
...,...,...,...
4995,23423,"“One Hundred and Five, North Tower.”",0
4996,12898,"“I made the patient swallow, with great diffic...",0
4997,16349,'What am I to do?' said odin. 'I should not le...,0
4998,41697,"'No!' cried the doctor, passionately. 'As He i...",0


In [49]:
x1 = X[X['author'] == 1].reset_index(drop=True)
x11 = x1.sample(n=5000, random_state=13).reset_index(drop=True)
x11

,text,author
0,it was bad very bad in me against such a creat...,1
1,on ascending the stairs the miss odins found s...,1
2,did you say that you had something to tell me sir,1
3,nay i had it from you you wrote me word of it ...,1
4,oh god her father and mother,1
...,...,...
4995,i should have thought said anne that my manner...,1
4996,such expressions assisted as they were by ever...,1
4997,of course said odin you will mention our seein...,1
4998,i will not oppose such kind politeness but i s...,1


In [50]:
x2 = X[X['author'] == 2].reset_index(drop=True)
x22 = x2.sample(n=5000, random_state=13).reset_index(drop=True)
x22

,text,author
0,i have had a note from someone who has,2
1,i shall glance into the case for you said odin...,2
2,no she is in a deep sleep,2
3,it seems that it was she had the surest inform...,2
4,well then supposing that c c h does stand for...,2
...,...,...
4995,that which my son himself indicated that of th...,2
4996,diverting the stream so i thought it best to e...,2
4997,just walk on a little way dear said mrs barcla...,2
4998,there was this dead man lying upon the sofa so...,2


In [51]:
x3 = X[X['author'] == 3].reset_index(drop=True)
x33 = x3.sample(n=5000, random_state=13).reset_index(drop=True)
x33

,text,author
0,why odin surely this isnt the first youve hear...,3
1,odin sat down odin sat down beside her,3
2,i know you are in a hurry to get to the prison...,3
3,you ve said so to yourself many times when you...,3
4,perhaps i ve been unfair to myself he observed...,3
...,...,...
4995,yes she is inquisitive assented the prince,3
4996,i was not delirious i knew what i was doing he...,3
4997,odin looked at him quickly,3
4998,i really came to sell an estate and i am in th...,3


In [52]:
x4 = X[X['author'] == 4].reset_index(drop=True)
x44 = x4.sample(n=5000, random_state=13).reset_index(drop=True)
x44

,text,author
0,can ye so indeed asked the jacobite well sir ...,4
1,we could stand a siege i said at length,4
2,good evening dance says the doctor with a nod ...,4
3,no sir said the boy,4
4,it will brisken you up says he,4
...,...,...
4995,ye are fair he said at last and as they tell m...,4
4996,i ask your pardon said the odin to a man of m...,4
4997,most certainly he answered with a smile let m...,4
4998,sir knight observed the messenger with bittern...,4


In [53]:
x5000 = pd.concat([x00, x11, x22, x33, x44])
x5000

,index,text,author
0,31768.0,"The suitor, kissing the hem of the garment aga...",0
1,21061.0,"“At,” repeated Mr. odin, still looking at the ...",0
2,10750.0,"“Dear, dear! Give it me back, Mum,” said Flops...",0
3,47317.0,"‘Be calm, my dear ma’am,’ said Mr. Chillip, in...",0
4,16917.0,Because he had voluntarily relinquished a titl...,0
...,...,...,...
4995,NaN,ye are fair he said at last and as they tell m...,4
4996,NaN,i ask your pardon said the odin to a man of m...,4
4997,NaN,most certainly he answered with a smile let m...,4
4998,NaN,sir knight observed the messenger with bittern...,4


In [54]:
x5000.reset_index(drop=True)

,index,text,author
0,31768.0,"The suitor, kissing the hem of the garment aga...",0
1,21061.0,"“At,” repeated Mr. odin, still looking at the ...",0
2,10750.0,"“Dear, dear! Give it me back, Mum,” said Flops...",0
3,47317.0,"‘Be calm, my dear ma’am,’ said Mr. Chillip, in...",0
4,16917.0,Because he had voluntarily relinquished a titl...,0
...,...,...,...
24995,NaN,ye are fair he said at last and as they tell m...,4
24996,NaN,i ask your pardon said the odin to a man of m...,4
24997,NaN,most certainly he answered with a smile let m...,4
24998,NaN,sir knight observed the messenger with bittern...,4


In [55]:
X_train, X_test, y_train, y_test = train_test_split(x5000.text, x5000.author, test_size=0.2, random_state=13)

### 벡터라이즈

#### tfidfvectorizer

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

##### GridSearchCV로 LogisticRegression C 하이퍼 파라미터 튜닝

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.672


In [58]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. CV는 3 Fold셋으로 설정.
params = { 'C': [0.01, 0.1, 1, 5, 10], 'random_state': [13]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print('Logistic Regression best C parameter:', grid_cv_lr.best_params_)

# 최적 C 값으로 학습된 grid_cv로 예측 수행하고 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.1min finished


Logistic Regression best C parameter: {'C': 10, 'random_state': 13}
TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.699


In [59]:
grid_cv_lr.best_params_

{'C': 10, 'random_state': 13}

In [60]:
grid_cv_lr.best_score_

0.6726502258745448

## 모델 하이퍼파라미터 튜닝

### MultinomialNB

In [61]:
from sklearn.naive_bayes import MultinomialNB

params = {'alpha': [0.01, 0.1, 0.5, 1.0],
         'fit_prior': ['True', 'False']}
clf = MultinomialNB()
grid_cv = GridSearchCV(clf, param_grid=params, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_cv.fit(X_train_tfidf_vect, y_train)
print('MultinomialNB best parameters:', grid_cv.best_params_)
print('MultinomialNB best accuracy score:', grid_cv.best_score_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


MultinomialNB best parameters: {'alpha': 0.1, 'fit_prior': 'True'}
MultinomialNB best accuracy score: 0.7032502035264274


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    0.8s finished


In [62]:
clf = MultinomialNB(alpha=0.1, fit_prior='True')
clf.fit(X_train_tfidf_vect, y_train)

train_pred = clf.predict(X_train_tfidf_vect)
test_pred = clf.predict(X_test_tfidf_vect)

print('MultinomialNB train accuracy score', accuracy_score(y_train, train_pred))
print('MultinomialNB test accuracy score', accuracy_score(y_test, test_pred))

MultinomialNB train accuracy score 0.97955
MultinomialNB test accuracy score 0.707


In [63]:
from sklearn.model_selection import StratifiedKFold

skfold = StratifiedKFold(n_splits=5)

In [64]:
from sklearn.model_selection import cross_validate
cross_validate(clf, X_train_tfidf_vect, y_train, scoring=None, cv=skfold, return_train_score=True)

{'fit_time': array([0.04706383, 0.04076099, 0.03973126, 0.03557205, 0.03543663]),
 'score_time': array([0.00729394, 0.00839114, 0.0065558 , 0.0066781 , 0.00654912]),
 'test_score': array([0.7025 , 0.7145 , 0.71225, 0.73025, 0.70875]),
 'train_score': array([0.979625 , 0.98125  , 0.9823125, 0.9821875, 0.9805   ])}

### LinearSVC

In [65]:
import sklearn
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

svm_clf = LinearSVC(C=1, loss='hinge', random_state=13)
svm_clf.fit(X_train_tfidf_vect, y_train)

train_pred = svm_clf.predict(X_train_tfidf_vect)
test_pred = svm_clf.predict(X_test_tfidf_vect)

print('LinearSVC train accuracy score:', accuracy_score(y_train, train_pred))
print('LinearSVC test accuracy score:', accuracy_score(y_test, test_pred))

LinearSVC train accuracy score: 0.9828
LinearSVC test accuracy score: 0.7074


In [66]:
params = {
    'C': [0.001, 0.01, 1], 
    'loss': ['squared_hinge', 'hinge'], 
    'penalty': ['l1', 'l2'], 
    'random_state': [13]
    }

grid_cv = GridSearchCV(LinearSVC(), param_grid=params, refit=True, verbose=2)
grid_cv.fit(X_train_tfidf_vect, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] C=0.001, loss=squared_hinge, penalty=l1, random_state=13 ........
[CV]  C=0.001, loss=squared_hinge, penalty=l1, random_state=13, total=   0.0s
[CV] C=0.001, loss=squared_hinge, penalty=l1, random_state=13 ........
[CV]  C=0.001, loss=squared_hinge, penalty=l1, random_state=13, total=   0.0s
[CV] C=0.001, loss=squared_hinge, penalty=l1, random_state=13 ........
[CV]  C=0.001, loss=squared_hinge, penalty=l1, random_state=13, total=   0.0s
[CV] C=0.001, loss=squared_hinge, penalty=l1, random_state=13 ........
[CV]  C=0.001, loss=squared_hinge, penalty=l1, random_state=13, total=   0.0s
[CV] C=0.001, loss=squared_hinge, penalty=l1, random_state=13 ........
[CV]  C=0.001, loss=squared_hinge, penalty=l1, random_state=13, total=   0.0s
[CV] C=0.001, loss=squared_hinge, penalty=l2, random_state=13 ........


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  C=0.001, loss=squared_hinge, penalty=l2, random_state=13, total=   0.2s
[CV] C=0.001, loss=squared_hinge, penalty=l2, random_state=13 ........
[CV]  C=0.001, loss=squared_hinge, penalty=l2, random_state=13, total=   0.2s
[CV] C=0.001, loss=squared_hinge, penalty=l2, random_state=13 ........
[CV]  C=0.001, loss=squared_hinge, penalty=l2, random_state=13, total=   0.2s
[CV] C=0.001, loss=squared_hinge, penalty=l2, random_state=13 ........
[CV]  C=0.001, loss=squared_hinge, penalty=l2, random_state=13, total=   0.2s
[CV] C=0.001, loss=squared_hinge, penalty=l2, random_state=13 ........
[CV]  C=0.001, loss=squared_hinge, penalty=l2, random_state=13, total=   0.2s
[CV] C=0.001, loss=hinge, penalty=l1, random_state=13 ................
[CV] . C=0.001, loss=hinge, penalty=l1, random_state=13, total=   0.0s
[CV] C=0.001, loss=hinge, penalty=l1, random_state=13 ................
[CV] . C=0.001, loss=hinge, penalty=l1, random_state=13, total=   0.0s
[CV] C=0.001, loss=hinge, penalty=l1, rand

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   18.6s finished


GridSearchCV(estimator=LinearSVC(),
             param_grid={'C': [0.001, 0.01, 1],
                         'loss': ['squared_hinge', 'hinge'],
                         'penalty': ['l1', 'l2'], 'random_state': [13]},
             verbose=2)

In [67]:
print('LinearSVC Best parameters:', grid_cv.best_params_)
print('LinearSVC Best accruacy score:', grid_cv.best_score_)

LinearSVC Best parameters: {'C': 1, 'loss': 'hinge', 'penalty': 'l2', 'random_state': 13}
LinearSVC Best accruacy score: 0.69375


In [68]:
svm_clf = LinearSVC(C=1, loss='hinge', penalty='l2', random_state=13)
svm_clf.fit(X_train_tfidf_vect, y_train)

train_pred = svm_clf.predict(X_train_tfidf_vect)
test_pred = svm_clf.predict(X_test_tfidf_vect)

print('LinearSVC train accuracy score:', accuracy_score(y_train, train_pred))
print('LinearSVC test accuracy score:', accuracy_score(y_test, test_pred)) 

LinearSVC train accuracy score: 0.9828
LinearSVC test accuracy score: 0.7074


In [69]:
svm_clf.decision_function(X_test_tfidf_vect) > 0

array([[False, False, False, False, False],
       [False, False,  True, False, False],
       [False, False, False, False, False],
       ...,
       [False, False, False, False, False],
       [False, False, False,  True, False],
       [False, False, False, False, False]])

In [70]:
# 교차검증

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=13)

In [71]:
from sklearn.model_selection import cross_validate, cross_val_score

cross_validate(svm_clf, X_train_tfidf_vect, y_train, scoring=None, cv=skf, return_train_score=True)

{'fit_time': array([2.54319715, 3.78981185, 1.85143685, 1.68291616, 1.63446403,
        2.37551904, 2.37976336, 1.35360193, 4.16437888, 2.33203411]),
 'score_time': array([0.00189495, 0.00179315, 0.00183797, 0.00183916, 0.00190902,
        0.00183392, 0.00191593, 0.00184321, 0.00187922, 0.00201392]),
 'test_score': array([0.7105, 0.709 , 0.7045, 0.7015, 0.7005, 0.704 , 0.699 , 0.712 ,
        0.687 , 0.707 ]),
 'train_score': array([0.98316667, 0.98372222, 0.9835    , 0.98355556, 0.98355556,
        0.98316667, 0.98283333, 0.98322222, 0.983     , 0.98305556])}

### SGDClassifier

In [72]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=13, loss='modified_huber')
sgd_clf.fit(X_train_tfidf_vect, y_train)

train_pred = sgd_clf.predict(X_train_tfidf_vect)
test_pred = sgd_clf.predict(X_test_tfidf_vect)

print('SGDClassifier train accuracy score:', accuracy_score(y_train, train_pred))
print('SGDClassifier test accuracy score:', accuracy_score(y_test, test_pred))

SGDClassifier train accuracy score: 0.98795
SGDClassifier test accuracy score: 0.7052


In [73]:
# 교차 검증

from sklearn.model_selection import cross_val_score

cross_val_score(sgd_clf, X_train_tfidf_vect, y_train, cv=5, scoring='accuracy')

array([0.68625, 0.69025, 0.691  , 0.70075, 0.69875])

In [74]:
# 오차 행렬

from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train_tfidf_vect, y_train, cv=3)

In [75]:
from sklearn.metrics import classification_report, confusion_matrix
confusion_matrix(y_train, y_train_pred)

array([[2441,  381,  430,  383,  365],
       [ 215, 2917,  422,  267,  164],
       [ 239,  262, 2638,  430,  438],
       [ 186,  254,  439, 2886,  213],
       [ 307,  189,  509,  295, 2730]])

In [76]:
params = {
    'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3],
    'loss': ['log', 'modified_huber', 'hinge', 'squared_hinge', 'perceptron'],
    'penalty': ['l2', 'elasticnet'],
    'n_jobs': [-1],
    'random_state': [13],
}

grid_cv = GridSearchCV(sgd_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_cv.fit(X_train_tfidf_vect, y_train)

Fitting 3 folds for each of 80 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   39.2s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:   45.3s finished


GridSearchCV(cv=3,
             estimator=SGDClassifier(loss='modified_huber', random_state=13),
             n_jobs=-1,
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                                   1000.0],
                         'loss': ['log', 'modified_huber', 'hinge',
                                  'squared_hinge', 'perceptron'],
                         'n_jobs': [-1], 'penalty': ['l2', 'elasticnet'],
                         'random_state': [13]},
             scoring='accuracy', verbose=1)

In [77]:
print('SGDClassifier best parameters:', grid_cv.best_params_)
print('SGDClassifier best accuracy score:', grid_cv.best_score_)

SGDClassifier best parameters: {'alpha': 0.0001, 'loss': 'modified_huber', 'n_jobs': -1, 'penalty': 'l2', 'random_state': 13}
SGDClassifier best accuracy score: 0.680600098407921


In [78]:
from sklearn.model_selection import cross_validate

cross_validate(sgd_clf, X_train_tfidf_vect, y_train, scoring=None, cv=skfold, return_train_score=True)

{'fit_time': array([0.20679712, 0.15851021, 0.15239787, 0.15211701, 0.15142894]),
 'score_time': array([0.01052999, 0.00664711, 0.0065608 , 0.00659204, 0.00652194]),
 'test_score': array([0.68625, 0.69025, 0.691  , 0.70075, 0.69875]),
 'train_score': array([0.9900625, 0.9903125, 0.9909375, 0.9903125, 0.9894375])}

### RidgeClassifier

In [79]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV, SGDClassifier
rd_clf = RidgeClassifier()
params = {
    'alpha': [0.01, 0.1, 0.5, 1.0],
    'normalize': [True, False],
    'max_iter': [100, 300],
    'random_state': [13]
}

grid_cv = GridSearchCV(rd_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_cv.fit(X_train_tfidf_vect, y_train)
print('RidgeClassifier best parameters:', grid_cv.best_params_)
print('RidgeClassifier best accuracy score:', grid_cv.best_score_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  1.2min finished


RidgeClassifier best parameters: {'alpha': 1.0, 'max_iter': 100, 'normalize': False, 'random_state': 13}
RidgeClassifier best accuracy score: 0.680850093408921


In [80]:
rd_clf = RidgeClassifier(alpha=1.0, max_iter=100, normalize='False', random_state=13)
rd_clf.fit(X_train_tfidf_vect, y_train)

train_pred = rd_clf.predict(X_train_tfidf_vect)
test_pred = rd_clf.predict(X_test_tfidf_vect)
print('RidgeClassifier best train accuracy score:', accuracy_score(y_train, train_pred))
print('RidgeClassifier best test accuracy score:', accuracy_score(y_test, test_pred))

RidgeClassifier best train accuracy score: 0.99105
RidgeClassifier best test accuracy score: 0.6652


In [81]:
cross_validate(rd_clf, X_train_tfidf_vect, y_train, scoring=None, cv=skf, return_train_score=True)

{'fit_time': array([0.97665191, 0.93324637, 1.04113579, 0.85441589, 0.9586122 ,
        0.95701599, 0.92036486, 0.94206595, 0.88842106, 0.97109294]),
 'score_time': array([0.00582409, 0.00579095, 0.00533104, 0.00541329, 0.00524712,
        0.00526905, 0.00533319, 0.00536704, 0.00529003, 0.00534225]),
 'test_score': array([0.6555, 0.667 , 0.66  , 0.655 , 0.651 , 0.674 , 0.6565, 0.6585,
        0.6555, 0.6615]),
 'train_score': array([0.99183333, 0.99211111, 0.99127778, 0.99177778, 0.99138889,
        0.99138889, 0.99094444, 0.99177778, 0.99188889, 0.99122222])}

### LogisticRegression

In [82]:
clf = LogisticRegression()
params = {
    'C': [0.01, 0.1, 0.5, 1.0],
    'max_iter': [100, 200, 500],
}

grid_cv = GridSearchCV(clf, param_grid=params, scoring='accuracy', verbose=1, n_jobs=-1)
grid_cv.fit(X_train_tfidf_vect, y_train)
print('LogisticRegression best parameters:', grid_cv.best_params_)
print('LogisticRegression best accuracy score:', grid_cv.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 10.0min finished


LogisticRegression best parameters: {'C': 1.0, 'max_iter': 100}
LogisticRegression best accuracy score: 0.6667500000000001


In [83]:
lr_clf = LogisticRegression(C=1.0, max_iter=500)
lr_clf.fit(X_train_tfidf_vect, y_train)

train_pred = lr_clf.predict(X_train_tfidf_vect)
test_pred = lr_clf.predict(X_test_tfidf_vect)

print('LogisticRegression train accuracy score:', accuracy_score(y_train, train_pred))
print('LogisticRegression test accuracy score:', accuracy_score(y_test, test_pred))

LogisticRegression train accuracy score: 0.93845
LogisticRegression test accuracy score: 0.6714


### DecisionTreeClassifier

In [84]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train_tfidf_vect, y_train)

train_pred = dt_clf.predict(X_train_tfidf_vect)
test_pred = dt_clf.predict(X_test_tfidf_vect)

print('DecisionTreeClassifier train score :', accuracy_score(y_train, train_pred))
print('DecisionTreeClassifier test score :', accuracy_score(y_test, test_pred))

DecisionTreeClassifier train score : 0.99305
DecisionTreeClassifier test score : 0.4332


In [85]:
params = {
    'max_depth': [5, 10, 20, 120],
    'min_samples_split': [16, 24],
    'min_samples_leaf': [16, 32],
    'random_state': [13]
}

grid_cv = GridSearchCV(dt_clf, param_grid=params, verbose=1, n_jobs=-1, cv=5, scoring='accuracy')
grid_cv.fit(X_train_tfidf_vect, y_train)

print('DecisionTreeClassifier best parameters:', grid_cv.best_params_)
print('DecisionTreeClassifier best accuracy score:', grid_cv.best_score_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  3.1min finished


DecisionTreeClassifier best parameters: {'max_depth': 120, 'min_samples_leaf': 16, 'min_samples_split': 16, 'random_state': 13}
DecisionTreeClassifier best accuracy score: 0.41059999999999997


## 모델 검증

In [86]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5)
kf = SGDClassifier(random_state=13, loss='modified_huber')

In [87]:
for train_idx, test_idx in kfold.split(X):
    print(len(train_idx), len(test_idx))

43903 10976
43903 10976
43903 10976
43903 10976
43904 10975


In [88]:
# 교차 검증 구현하기

kf = KFold(n_splits=2)

print(kf.get_n_splits(X_train_tfidf_vect))
print(kf)
for train_idx, test_idx in kf.split(X_train_tfidf_vect):
    print('--- idx')
    print(train_idx, test_idx)
    print('--- train data')
    print(X_train_tfidf_vect)
    print('--- val data')
    print(X[test_idx])

2
KFold(n_splits=2, random_state=None, shuffle=False)
--- idx
[10000 10001 10002 ... 19997 19998 19999] [   0    1    2 ... 9997 9998 9999]
--- train data
  (0, 50906)	0.3287746372510267
  (0, 77001)	0.3287746372510267
  (0, 77005)	0.3611880139840538
  (0, 89796)	0.25611092461001495
  (0, 89802)	0.3611880139840538
  (0, 160880)	0.14511542732327234
  (0, 160950)	0.16086330799095613
  (0, 168311)	0.06387958773121784
  (0, 169734)	0.3366682410414953
  (0, 203684)	0.16192306553866398
  (0, 203822)	0.2978053204013427
  (0, 271871)	0.24028745402761625
  (0, 271883)	0.3468448662864598
  (1, 380)	0.0837787627028299
  (1, 390)	0.10817336061386297
  (1, 937)	0.10817336061386297
  (1, 938)	0.10817336061386297
  (1, 9816)	0.059544285309588295
  (1, 9826)	0.10817336061386297
  (1, 11584)	0.06705036871881494
  (1, 11650)	0.10817336061386297
  (1, 15078)	0.09011251049531985
  (1, 15079)	0.10817336061386297
  (1, 23205)	0.07129157393440207
  (1, 23234)	0.10817336061386297
  :	:
  (19998, 160672)	0.086

KeyError: "None of [Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,\n            ...\n            9990, 9991, 9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999],\n           dtype='int64', length=10000)] are in the [columns]"

In [ ]:
from sklearn.model_selection import cross_val_score

skfold = StratifiedKFold(n_splits=5)
cross_val_score(sgd_clf, X_train_tfidf_vect, y_train, cv=skfold, scoring=None)

In [ ]:
cross_val_score(sgd_clf, X_train_tfidf_vect, y_train, cv=5, scoring='accuracy')

In [ ]:
# train score 와 같이

from sklearn.model_selection import cross_validate

cross_validate(sgd_clf, X_train_tfidf_vect, y_train, scoring=None, cv=skfold, return_train_score=True)

In [ ]:
test_x.text

In [ ]:
X_train

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환.
tfidf_vect_test = TfidfVectorizer(stop_words='english', max_features=30000)
tfidf_vect_test.fit(test_x.text)
X_train_tfidf_test = tfidf_vect_test.transform(test_x.text)

In [ ]:
pred

In [ ]:
pred = lr_clf.predict(X_train_tfidf_test)

In [ ]:
pred

In [ ]:
test_x['author'] = pred

In [ ]:
test_x